In [1]:
import pandas as pd
import numpy as np
from optuna.pruners import HyperbandPruner
from optuna.samplers._tpe.sampler import TPESampler
from sklearn.model_selection import KFold, train_test_split
from lohrasb.best_estimator import BaseModel
from sklearn.pipeline import Pipeline
from feature_engine.imputation import (
    MeanMedianImputer
    )
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    f1_score)
from sklearn.metrics import f1_score, make_scorer
from catboost import CatBoostClassifier

2023-07-04 11:32:12,813 :: matplotlib :: matplotlib data path: /Users/hjavedani/Documents/Lohrasb/.venv/lib/python3.10/site-packages/matplotlib/mpl-data
2023-07-04 11:32:12,820 :: matplotlib :: CONFIGDIR=/Users/hjavedani/.matplotlib
2023-07-04 11:32:12,822 :: matplotlib :: interactive is False
2023-07-04 11:32:12,823 :: matplotlib :: platform is darwin
2023-07-04 11:32:12,839 :: graphviz._tools :: deprecate positional args: graphviz.backend.piping.pipe(['renderer', 'formatter', 'neato_no_op', 'quiet'])
2023-07-04 11:32:12,840 :: graphviz._tools :: deprecate positional args: graphviz.backend.rendering.render(['renderer', 'formatter', 'neato_no_op', 'quiet'])
2023-07-04 11:32:12,843 :: graphviz._tools :: deprecate positional args: graphviz.backend.unflattening.unflatten(['stagger', 'fanout', 'chain', 'encoding'])
2023-07-04 11:32:12,844 :: graphviz._tools :: deprecate positional args: graphviz.backend.viewing.view(['quiet'])
2023-07-04 11:32:12,848 :: graphviz._tools :: deprecate positio

#### Example: Audiology (Standardized) Data Set
#### https://archive.ics.uci.edu/ml/datasets/Audiology+%28Standardized%29


In [2]:
urldata = "https://archive.ics.uci.edu/ml/machine-learning-databases/lymphography/lymphography.data"
urlname = "https://archive.ics.uci.edu/ml/machine-learning-databases/lung-cancer/lung-cancer.names"
# column names
col_names = [
    "class",
    "lymphatics",
    "block of affere",
    "bl. of lymph. c",
    "bl. of lymph. s",
    "by pass",
    "extravasates",
    "regeneration of",
    "early uptake in",
    "lym.nodes dimin",
    "lym.nodes enlar",
    "changes in lym.",
    "defect in node",
    "changes in node",
    "special forms",
    "dislocation of",
    "exclusion of no",
    "no. of nodes in",

]
data = pd.read_csv(urldata,names=col_names)
data.head()

,class,lymphatics,block of affere,bl. of lymph. c,bl. of lymph. s,by pass,extravasates,regeneration of,early uptake in,lym.nodes dimin,lym.nodes enlar,changes in lym.,defect in node,changes in node,special forms,dislocation of,exclusion of no,no. of nodes in
3,4,2,1,1,1,1,1,2,1,2,2,2,4,8,1,1,2,2
2,3,2,1,1,2,2,1,2,1,3,3,2,3,4,2,2,2,2
3,3,2,2,2,2,2,2,2,1,4,3,3,4,8,3,2,2,7
3,3,1,1,1,1,2,1,2,1,3,3,4,4,4,3,1,2,6
2,3,1,1,1,1,1,1,1,1,2,2,4,3,5,1,2,2,1


#### Define labels and train-test split


In [3]:

data.loc[(data["class"] == 1) | (data["class"] == 2), "class"] = 0
data.loc[data["class"] == 3, "class"] = 1
data.loc[data["class"] == 4, "class"] = 2
data["class"] = data["class"].astype(int)

# Train test split

X = data.loc[:, data.columns != "class"]
y = data.loc[:, data.columns == "class"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33,  random_state=42
)



#### Define estimator and set its arguments 


In [4]:
estimator = CatBoostClassifier()
estimator_params = {
        "depth": [4, 5],
        "learning_rate": [0.01, 0.1],

    }

kwargs = {  
            # extra params for model if any
            'main_random_kwargs':{},
            # params for fit method or fit_params 
            'fit_grid_kwargs' :{
            'sample_weight':None,
            },
            # params for RandomSearchCV 
            'random_search_kwargs' : {
            'estimator':estimator,
            'param_distributions':estimator_params,
            'scoring' :'f1_weighted',
            'verbose':3,
            'n_jobs':-1,
            'cv':KFold(2),
            'refit':True,
            }
            }
    

#### Define BestModel using random search

In [5]:

obj = BaseModel().optimize_by_randomsearchcv(
    kwargs=kwargs
        )

#### Capture int features (the problem has only integer features)

In [6]:
int_cols = X_train.select_dtypes(include=["int"]).columns.tolist()
print(int_cols)

['lymphatics', 'block of affere', 'bl. of lymph. c', 'bl. of lymph. s', 'by pass', 'extravasates', 'regeneration of', 'early uptake in', 'lym.nodes dimin', 'lym.nodes enlar', 'changes in lym.', 'defect in node', 'changes in node', 'special forms', 'dislocation of', 'exclusion of no', 'no. of nodes in']


#### Build sklearn pipeline

In [7]:


pipeline =Pipeline([
            # int missing values imputers
            ('intimputer', MeanMedianImputer(
                imputation_method='median', variables=int_cols)),
            # classification model
            ('obj', obj)

 ])


#### Run Pipeline

In [8]:
pipeline.fit(X_train,y_train)
y_pred = pipeline.predict(X_test)


Fitting 2 folds for each of 4 candidates, totalling 8 fits


/Users/hjavedani/Documents/Lohrasb/.venv/lib/python3.10/site-packages/sklearn/model_selection/_search.py:306: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


0:	learn: 1.0964066	total: 51ms	remaining: 51s
1:	learn: 1.0938873	total: 51.9ms	remaining: 25.9s
2:	learn: 1.0917111	total: 52.6ms	remaining: 17.5s
3:	learn: 1.0894544	total: 53.2ms	remaining: 13.2s
4:	learn: 1.0873726	total: 53.8ms	remaining: 10.7s
5:	learn: 1.0856279	total: 54.3ms	remaining: 8.99s
6:	learn: 1.0825365	total: 55.2ms	remaining: 7.83s
0:	learn: 1.0964765	total: 51.2ms	remaining: 51.2s
0:	learn: 1.0769296	total: 50ms	remaining: 49.9s
7:	learn: 1.0801677	total: 55.7ms	remaining: 6.91s
1:	learn: 1.0940183	total: 51.9ms	remaining: 25.9s
8:	learn: 1.0782821	total: 56.2ms	remaining: 6.19s
1:	learn: 1.0533784	total: 50.8ms	remaining: 25.4s
2:	learn: 1.0925400	total: 52.3ms	remaining: 17.4s
9:	learn: 1.0759317	total: 56.6ms	remaining: 5.61s
3:	learn: 1.0910035	total: 52.9ms	remaining: 13.2s
2:	learn: 1.0343271	total: 51.4ms	remaining: 17.1s
10:	learn: 1.0740978	total: 57.2ms	remaining: 5.14s
4:	learn: 1.0888035	total: 53.4ms	remaining: 10.6s
3:	learn: 1.0146348	total: 52ms	rema

#### Check performance of the pipeline

In [9]:
print('F1 score : ')
print(f1_score(y_test,y_pred, average='weighted'))
print('Classification report : ')
print(classification_report(y_test,y_pred))
print('Confusion matrix : ')
print(confusion_matrix(y_test,y_pred))


F1 score : 
0.4419742365031119
Classification report : 
              precision    recall  f1-score   support

           0       0.46      0.57      0.51        21
           1       0.38      0.43      0.40        14
           2       0.57      0.29      0.38        14

    accuracy                           0.45        49
   macro avg       0.47      0.43      0.43        49
weighted avg       0.47      0.45      0.44        49

Confusion matrix : 
[[12  7  2]
 [ 7  6  1]
 [ 7  3  4]]
